In [40]:
# Install AIF360
!pip install 'aif360[lime]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=e9e52219d8f9d990fe523a3d3e362dd87c9574c53650be1c7bd5ec7e2ec35a99
  Stored in directory: /root/.cache/pip/wheels/e7/5d/0e/4b4fff9a47468fed5633211fb3b76d1db43fe806a17fb7486a
Successfully built lime


In [41]:
import os
import urllib
import aif360

# Obtain the location where it is installed
LIB_PATH = aif360.__file__.rsplit("aif360", 1)[0]

# check if the data got download properly
def check_data_or_download(destn, files, data_source_directory):
    check = all(item in os.listdir(destn) for item in files)
    if check:
        print("Adult dataset is available for us")
    else:
        print("Some files are missing. Downloading now.")
        for data_file in files:
            _ = urllib.request.urlretrieve(data_source_directory + data_file,
                                           os.path.join(destn, data_file))

# Download adult dataset
data_source_directory = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/"
destn = os.path.join(LIB_PATH, "aif360", "data", "raw", "adult")
files = ["adult.data", "adult.test", "adult.names"]

check_data_or_download(destn, files, data_source_directory)

Some files are missing. Downloading now.


In [42]:
from __future__ import print_function

%matplotlib inline

import sklearn.model_selection
import sklearn.metrics
import sklearn.datasets
import sklearn.ensemble
import sklearn.preprocessing
import numpy as np
import lime
import lime.lime_tabular
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
import sys
sys.path.append("../")
import numpy as np
from aif360.datasets import BinaryLabelDataset
from aif360.metrics.binary_label_dataset_metric import BinaryLabelDatasetMetric
from aif360.metrics.classification_metric import ClassificationMetric
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
from aif360.algorithms.preprocessing.reweighing import Reweighing



from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

from aif360.datasets.lime_encoder import LimeEncoder


from aif360.datasets.adult_dataset import AdultDataset

from aif360.metrics import BinaryLabelDatasetMetric

In [52]:
# 2️⃣ Cargar dataset Adult (predice si ingreso anual >50k)
# El dataset Adult es usado para predecir si una persona gana
# más de 50,000 USD al año (>50K) o no (<=50K), usando atributos socioeconómicos.
# https://aif360.readthedocs.io/en/v0.2.3/modules/standard_datasets.html#adult-dataset

dataset = AdultDataset()
df, metadata = dataset.convert_to_dataframe()
df.head(20)

,age,education-num,race,sex,capital-gain,capital-loss,hours-per-week,workclass=Federal-gov,workclass=Local-gov,workclass=Private,...,native-country=Puerto-Rico,native-country=Scotland,native-country=South,native-country=Taiwan,native-country=Thailand,native-country=Trinadad&Tobago,native-country=United-States,native-country=Vietnam,native-country=Yugoslavia,income-per-year
0,25.0,7.0,0.0,1.0,0.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,38.0,9.0,1.0,1.0,0.0,0.0,50.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,28.0,12.0,1.0,1.0,0.0,0.0,40.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,44.0,10.0,0.0,1.0,7688.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
5,34.0,6.0,1.0,1.0,0.0,0.0,30.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,63.0,15.0,1.0,1.0,3103.0,0.0,32.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
8,24.0,10.0,1.0,0.0,0.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,55.0,4.0,1.0,1.0,0.0,0.0,10.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10,65.0,9.0,1.0,1.0,6418.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
11,36.0,13.0,1.0,1.0,0.0,0.0,40.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [54]:
# 3️⃣ Definir atributo protegido y grupo privilegiado
# sex=1 → Male (privileged), sex=0 → Female (unprivileged)
# un atributo "protected" es un atributo que no debería ser usado para discriminar (raza, genero, sexo, religión)
# un grupo "privileged" valores del atributo protegido (hombres -1- privilegiado, mujeres -2- no privilegiado)
protected_attribute = 'sex'
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

In [57]:
# 4️⃣ Calcular métricas de fairness
metric = BinaryLabelDatasetMetric(dataset,
                                  unprivileged_groups=unprivileged_groups,
                                  privileged_groups=privileged_groups)

#Disparate Impact (ratio) = P(Y = 1 | D = unprivileged) / P(Y = 1 | D = privileged)
# 1 (ok) | < 1 (sesgo) | > 1 (favorece unprivileged)
print("Disparate Impact (DI):", metric.disparate_impact())
#Paridad estadística Pr(Y = 1 | D = unprivileged - Pr(Y = 1 | D = privileged)
print("Mean Difference:", metric.mean_difference())

Disparate Impact (DI): 0.3634695423643793
Mean Difference: -0.198901432678815
